## **CNA English (Taiwan Focus)**

In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import json
from tqdm import tqdm
import time


headers = {
   'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'
}


start_date = datetime.strptime("2025-04-01", "%Y-%m-%d")
end_date = datetime.strptime("2025-05-12", "%Y-%m-%d")


results = []


num_days = (end_date - start_date).days + 1
total_tasks = num_days * (20 - 2 + 1)


with tqdm(total=total_tasks, desc="爬取進度", ncols=100) as pbar:
   current_date = start_date
   while current_date <= end_date:
       date_str = current_date.strftime("%Y%m%d")
       for i in range(2, 21):
           article_id = f"{i:04d}"
           full_id = f"{date_str}{article_id}"  
           url = f"https://focustaiwan.tw/cross-strait/{full_id}"
           try:
               response = requests.get(url, headers=headers, timeout=10)
               time.sleep(0.5)  


               if response.status_code != 200:
                   pbar.update(1)
                   continue


               soup = BeautifulSoup(response.text, "html.parser")


               title_tag = soup.select_one("span.h1t")
               date_tag = soup.select_one("div.updatetime")
               content_tags = soup.select("div.paragraph > p")


               if not (title_tag and date_tag and content_tags):
                   pbar.update(1)
                   continue


               title = title_tag.text.strip()
               date_raw = date_tag.text.strip()
               date_obj = datetime.strptime(date_raw.split(" ")[0], "%m/%d/%Y")
               date = date_obj.strftime("%Y-%m-%d")
               content = "\n".join(p.text.strip() for p in content_tags)


               results.append({
                   "media_name": "中央社",
                   "title": title,
                   "date": date,
                   "content": content
               })


           except Exception as e:
               pass 


           pbar.update(1)


       current_date += timedelta(days=1)




with open("中央社兩岸新聞.json", "w", encoding="utf-8") as f:
   json.dump(results, f, ensure_ascii=False, indent=2)


print(f"\n✅ 完成，共擷取 {len(results)} 篇新聞。")






爬取進度:   1%|▎                                                    | 4/798 [00:05<17:18,  1.31s/it]


KeyboardInterrupt: 

# **Pts news**

In [15]:
! /Applications/Python\ 3.13/Install\ Certificates.command
!pip3 isinstall --upgrade certifi
!pip3 install requests beautifulsoup4 tqdm
!pip3 install selenium webdriver-manager tqdm

 -- pip install --upgrade certifi
 -- removing any existing file or link
 -- creating symlink to certifi certificate bundle
 -- setting permissions
 -- update complete
ERROR: unknown command "isinstall" - maybe you meant "install"
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 9.4 MB 1.0 MB/s eta 0:00:01
     |████████████████████████████████| 499 kB 1.1 MB/s eta 0:00:01
  Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl (10 kB)
  Using cached sortedcontainers-2.4.0-py2.py3-none-any.whl (29 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl (24 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
You should consider upgrading via the '/A

In [22]:
# pip install selenium webdriver-manager tqdm

import re, json, time
from datetime import datetime
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

START, END = datetime(2025,4,1), datetime(2025,5,12)
CAT_URL     = "https://news.pts.org.tw/category/9?page={}"
ARTICLE_RE  = re.compile(r"/article/\d+$")

opt = webdriver.ChromeOptions()
opt.add_argument("--start-maximized")
opt.headless = False                          # 想隱藏改 True
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=opt)

article_urls = []
page = 1
while True:
    driver.get(CAT_URL.format(page))

    # 等 JS 把列表渲染好（偵測任何 <a> 符合 /article/123456）
    WebDriverWait(driver, 15).until(
        lambda d: d.execute_script(
            "return [...document.querySelectorAll('a')]"
            ".some(a=>a.getAttribute('href')?.match(/\\/article\\/\\d+$/))"))

    # ⬇︎ 把所有符合的 link 收集起來
    links = driver.execute_script("""
        return [...new Set(
          [...document.querySelectorAll('a')]
            .map(a=>a.href)
            .filter(h=>h && /\\/article\\/\\d+$/.test(h))
        )];
    """)
    if not links: break
    article_urls.extend(links)

    # 如果本頁最舊一篇都 still >= START，就翻下一頁；否則停
    dates_on_page = driver.execute_script("""
        return [...document.querySelectorAll('time, span, p')]
               .map(el=>el.textContent.match(/(\\d{4})[\\/.-](\\d{1,2})[\\/.-](\\d{1,2})/))
               .filter(Boolean).map(m=>m[0]);
    """)
    oldest = min([datetime.strptime(d, "%Y/%m/%d") for d in dates_on_page]
                 or [END])
    if oldest < START: break
    page += 1
    time.sleep(0.8)

# ---------- 逐篇抓正文 ----------
results = []
bar = tqdm(article_urls, desc="抓文章", unit="篇")
for url in bar:
    driver.get(url)
    try:
        WebDriverWait(driver, 12).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "meta[property='article:published_time']")))
    except:           # 某些連結被撤稿照跳
        continue

    iso = driver.find_element(By.CSS_SELECTOR,
          "meta[property='article:published_time']").get_attribute("content")
    pub_dt = datetime.fromisoformat(iso[:19])
    if not (START <= pub_dt <= END):
        continue

    title = driver.find_element(By.CSS_SELECTOR,
        "meta[property='og:title']").get_attribute("content").strip()
    paras = [p.text.strip() for p in driver.find_elements(By.CSS_SELECTOR,"article p") if p.text.strip()]
    content = "\n".join(paras)

    results.append({
        "media_name": "公視新聞網",
        "title"     : title,
        "date"      : pub_dt.strftime("%Y-%m-%d"),
        "content"   : content
    })
    time.sleep(0.25)

driver.quit()

with open("公視_兩岸新聞.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n✅ 抓完！共 {len(results)} 篇。")

抓文章: 100%|██████████| 90/90 [02:13<00:00,  1.49s/篇]


✅ 抓完！共 50 篇。
